<a href="https://colab.research.google.com/github/dotsnangles/NMT-with-transformers/blob/master/training_mT5-small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

### Set notebook parameters

In [ ]:
run_name = 'en2ko run on aws ec2 with NVIDIA A10G with the domain data'

project_name = 'en2ko-translator-mt5-small-with-the-domain-data'

num_train_epochs = 100
batch_size = 16
gradient_accumulation_steps = 1

learning_rate = 2e-5
weight_decay = 0.01

lr_scheduler_type = 'cosine'
warmup_ratio = 0.1

predict_with_generate = False
generation_max_length = 256

# early_stopping_patience = 5
save_total_limit = 5

load_best_model_at_end = True
metric_for_best_model='eval_loss'

save_strategy = "epoch"
evaluation_strategy = "epoch"
# save_steps = 1250
# eval_steps = 1250

logging_strategy = "steps"
logging_first_step = True 
logging_steps = 500

fp16 = False

### Prerequisites

In [ ]:
# !conda install -c conda-forge datasets transformers sentencepiece sacrebleu folium wandb pandas gdown jupyterlab ipywidgets papermill

In [ ]:
# import gdown
# id = "1J21-T8wYjlj-91CxtxEzrcE34CDt7CM3"
# gdown.download_folder(id=id, quiet=True, use_cookies=False)

### Set WandB 

In [ ]:
%env WANDB_NOTEBOOK_NAME=/home/ubuntu/codes/NMT-with-transformers/training_mT5_small_domain.ipynb
%env WANDB_PROJECT=$project_name
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all

In [ ]:
import wandb
wandb.login()

### Model Selection

In [ ]:
model_ckpt = 'google/mt5-small'

### Import stuff

In [ ]:
import pandas as pd
from datasets import Dataset, load_metric
from transformers import AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback

In [ ]:
train_df = pd.read_csv('./data/train_domain.csv')
val_df = pd.read_csv('./data/val_domain.csv')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_fast=False)

### Measure token length

In [ ]:
def measure_len(sample):
    return len(tokenizer.encode(sample))

In [ ]:
src_prefix = "translate English to Korean: "

print('length of src_prefix:', measure_len(src_prefix))
print(tokenizer.encode(src_prefix))
with tokenizer.as_target_tokenizer():
    print(tokenizer.encode(src_prefix))

In [ ]:
train_df_en_len = train_df['en'].apply(measure_len)
train_df_ko_len = train_df['ko'].apply(measure_len)
val_df_en_len = val_df['en'].apply(measure_len)
val_df_ko_len = val_df['ko'].apply(measure_len)

In [ ]:
max(train_df_en_len)+7, max(train_df_ko_len), max(val_df_en_len)+7, max(val_df_ko_len)

### df to ds

In [ ]:
train_ds = Dataset.from_pandas(train_df[['en', 'ko']])
val_ds = Dataset.from_pandas(val_df[['en', 'ko']])
# .shuffle(seed=42)[:val_ds_len]
# val_ds = Dataset.from_dict(val_ds)
train_ds, val_ds

In [ ]:
idx = 0
for e in train_ds:
    print(e)
    idx += 1
    if idx == 2:
        break

In [ ]:
idx = 0
for e in val_ds:
    print(e)
    idx += 1
    if idx == 2:
        break

### Preprocess

In [ ]:
source_lang = "en"
target_lang = "ko"
prefix = "translate English to Korean: "

def preprocess_function(examples):
    inputs = [prefix + example for example in examples[source_lang]]
    targets = [example for example in examples[target_lang]]
    model_inputs = tokenizer(inputs, max_length=160, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=160, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

#### Test preprocess_function

In [ ]:
train_ds[:3]

In [ ]:
preprocess_test = preprocess_function(train_ds[:3])
print('input id', preprocess_test.input_ids[0])
print(tokenizer.decode(preprocess_test.input_ids[0]), '\n')
print('attention mask', preprocess_test.attention_mask[0], '\n')
print('label', preprocess_test.labels[0])
print(tokenizer.decode(preprocess_test.labels[0]))

In [ ]:
tokenized_train = train_ds.map(preprocess_function, batched=True)
tokenized_val = val_ds.map(preprocess_function, batched=True)
tokenized_train, tokenized_val

### Load metric

In [ ]:
metric = load_metric("sacrebleu")

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

### Check and Load model

In [ ]:
config = AutoConfig.from_pretrained(model_ckpt)

In [ ]:
model = AutoModelForSeq2SeqLM.from_config(config)

In [ ]:
model_name = model_ckpt.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    report_to='wandb',
    run_name=run_name,

    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,

    learning_rate=learning_rate,
    weight_decay=weight_decay,

    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,

    # predict_with_generate=predict_with_generate,
    # generation_max_length=generation_max_length,

    save_total_limit=save_total_limit,

    load_best_model_at_end=load_best_model_at_end,
    metric_for_best_model=metric_for_best_model,
    
    save_strategy=save_strategy,
    evaluation_strategy=evaluation_strategy,
    # save_steps=save_steps,
    # eval_steps=eval_steps,

    logging_strategy=logging_strategy,
    logging_first_step=logging_first_step, 
    logging_steps=logging_steps,
    
    fp16=fp16,
)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# es = EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    # callbacks=[es],
)

In [ ]:
trainer.train()
wandb.finish()

In [ ]:
trainer.save_model('./save_model')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('./save_model')

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('./save_model')

In [ ]:
args = Seq2SeqTrainingArguments(
    'eval',
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=256,
)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    # train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate()